In [17]:
import pickle as pkl
import time
import os
import numpy as np
import sys
from music21 import instrument, note, stream, chord, duration
from RNNmodel import create_network
import matplotlib.pyplot as plt

In [18]:
with open("dictionary.pkl", "rb") as input_file:
    dictionary = pkl.load(input_file)


durations_to_int = dictionary["durations_to_int"]
int_to_duration = dictionary["int_to_duration"]
notes_to_int = dictionary["notes_to_int"]
int_to_notes = dictionary["int_to_notes"]
phrases_to_int = dictionary["phrases_to_int"]
int_to_phrases = dictionary["int_to_phrases"]

n_notes = len(notes_to_int)
n_durations = len(durations_to_int)
n_phrases = len(phrases_to_int)

In [19]:
weights_folder = 'weights'
weights_file = 'weights.h5'
embed_size = 100
rnn_units = 256
use_attention = True

model, att_model = create_network(n_notes, n_durations, n_phrases, embed_size, rnn_units, use_attention)

# Load the weights to each node
weight_source = os.path.join(weights_folder,weights_file)
model.load_weights(weight_source)
model.summary()

2023-05-08 10:45:06.613508: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-08 10:45:06.624838: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-08 10:45:06.627255: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding_2 (Embedding)        (None, None, 100)    4800        ['input_3[0][0]']                
                                                                                                  
 embedding_3 (Embedding)        (None, None, 100)    1900        ['input_4[0][0]']                
                                                                                            

In [20]:
import pickle
piece = None
with open('../data/data_2/data_germany_1.pkl', 'rb') as handle:
    piece = pickle.load(handle)

In [21]:
notes_input = [ notes_to_int[note] for note in piece[0][0] ]
print(notes_input)

[19, 24, 24, 28, 26, 24, 23, 21, 21, 21, 29, 23, 46, 31, 24, 46, 19, 19, 24, 24, 24, 24, 28, 26, 24, 23, 21, 21, 21, 29, 23, 46, 31, 24, 46, 19, 24, 24, 24, 24, 28, 26, 24, 21, 21, 21, 26, 24, 23, 23, 23, 23, 21, 23, 24, 46]


In [22]:
durations_input = [ durations_to_int[duration] for duration in piece[0][1] ]
print(durations_input)

[7, 7, 7, 4, 4, 4, 4, 7, 7, 7, 7, 10, 7, 7, 10, 7, 4, 4, 6, 2, 4, 4, 4, 4, 4, 4, 7, 7, 7, 7, 10, 7, 7, 10, 7, 7, 10, 7, 7, 10, 10, 10, 10, 12, 7, 10, 13, 10, 12, 7, 10, 10, 10, 10, 13, 7]


In [23]:
phrases_input = piece[0][2]
print(phrases_input)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1]


In [24]:
max_extra_notes = len(notes_input) - 1
seq_len = 8
phrases_output = []
phrases_output.append(0) # after the first note (that is already in the initial sequence)

notes_input_sequence = [notes_input[0]] # the first note element of the sequence
durations_input_sequence = [durations_input[0]] # the first duration element of the sequence

if seq_len is not None:
    notes_input_sequence = [notes_to_int[129]] * (seq_len - len(notes_input_sequence)) + notes_input_sequence # the sequence length must be always equal seq_len, therefore we fill the vector with values 129 (the 'START' token) at the beginning 
    durations_input_sequence = [0] * (seq_len - len(durations_input_sequence)) + durations_input_sequence # the same for durations

In [25]:
print(notes_input_sequence)

[47, 47, 47, 47, 47, 47, 47, 19]


In [26]:
print(durations_input_sequence)

[0, 0, 0, 0, 0, 0, 0, 7]


In [27]:
for note_index in range(max_extra_notes):

    prediction_input = [
        np.array([notes_input_sequence])
        , np.array([durations_input_sequence])
       ]

    phrases_prediction = model.predict(prediction_input, verbose=0)
    print(notes_input_sequence)
    print(durations_input_sequence)
    print(phrases_prediction[0][0])
    print(phrases_prediction[0][1])
    
    phrases_output.append(0 if phrases_prediction[0][0] >= phrases_prediction[0][1] else 1)

    notes_input_sequence.append(notes_input[note_index])
    durations_input_sequence.append(durations_input[note_index])
    
    if len(notes_input_sequence) > seq_len:
        notes_input_sequence = notes_input_sequence[1:]
        durations_input_sequence = durations_input_sequence[1:]

2023-05-08 10:45:09.186151: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-08 10:45:09.187424: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-08 10:45:09.190148: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

[47, 47, 47, 47, 47, 47, 47, 19]
[0, 0, 0, 0, 0, 0, 0, 7]
0.99998033
1.9714153e-05
[47, 47, 47, 47, 47, 47, 19, 19]
[0, 0, 0, 0, 0, 0, 7, 7]
0.99995923
4.077219e-05
[47, 47, 47, 47, 47, 19, 19, 24]
[0, 0, 0, 0, 0, 7, 7, 7]
0.99960655
0.00039343038
[47, 47, 47, 47, 19, 19, 24, 24]
[0, 0, 0, 0, 7, 7, 7, 7]
0.9999151
8.487423e-05
[47, 47, 47, 19, 19, 24, 24, 28]
[0, 0, 0, 7, 7, 7, 7, 4]
0.99987316
0.00012686968
[47, 47, 19, 19, 24, 24, 28, 26]
[0, 0, 7, 7, 7, 7, 4, 4]
0.9997477
0.00025233297
[47, 19, 19, 24, 24, 28, 26, 24]
[0, 7, 7, 7, 7, 4, 4, 4]
0.9997863
0.00021373609
[19, 19, 24, 24, 28, 26, 24, 23]
[7, 7, 7, 7, 4, 4, 4, 4]
0.99981457
0.00018543338
[19, 24, 24, 28, 26, 24, 23, 21]
[7, 7, 7, 4, 4, 4, 4, 7]
0.9999927
7.2813586e-06
[24, 24, 28, 26, 24, 23, 21, 21]
[7, 7, 4, 4, 4, 4, 7, 7]
0.9990676
0.0009324094
[24, 28, 26, 24, 23, 21, 21, 21]
[7, 4, 4, 4, 4, 7, 7, 7]
0.9999956
4.4613234e-06
[28, 26, 24, 23, 21, 21, 21, 29]
[4, 4, 4, 4, 7, 7, 7, 7]
0.9999882
1.1829627e-05
[26, 24, 23, 2

In [28]:
print(phrases_output)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1]


In [29]:
print(phrases_input) # should be

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1]


In [30]:
for i in range(len(notes_input)):
    if phrases_output[i] == 1:
        k = i+1
        while k < len(notes_input) and k > 1:
            phrases_input[k-1] = 0
            phrases_input[k] = 1
            if notes_input[k] != 128:
                break

In [31]:
print(phrases_output)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1]
